### Project 02: Create a GUI Notebook Program

Joseph Choi March 31 2024

Project 2 will adapt the procedural code we have been working on in INST326_SimpleGUI_Note_Form_IO.ipynb to create an OOP version of the program using three classes.

    A Notebook or MainWindow class
    A Form or TopWindow class
    A Note class

The MainWindow class is a subclass of Tkinter’s tk.Tk class and thus inherits its attributes and methods, while allowing us to customize the new window objects to our needs. These new window objects will represent “notebooks” or collections of notes, and allow us to work with those notes. (I have named it MainWindow because this class definition could be used to create any kind of main window in Tkinter. We are using it to represent notebooks in this application, but you might use it for other purposes in onther applications.)


The TopWindow class creates a new top window in Tkinter, which is essentially a form for entering the title, text, links, and tags for our note. When we submit the note, this “form” object has a method that creates the note’s metadata and a new Note object. That note object is appended to the list of all notes, which is an attribute of the notebook (MainWindow) class.
The Note class creates note objects that contain the  title, text, links, tags, and metadata for each note.

For Project 02 you will:  

    1. Create one notebook or MainWindow object  
    2. Create several (at least 3) ‘real’ notes for your final submission. By ‘real’ I mean actual notes about python that are useful to you. Print them in the cell at the bottom of the notebook.
    3. Save those notes to a single .txt, .csv, or .json file (your choice of format).  
    4. Retrieve those notes and 
    5. Display representations of them as either labels or buttons in the  main window (remember how we displayed cards in project 01). These representations are not whole notes. Rather, they are object representations of the notes.  
    6. When they are clicked the whole note pops up in a new window - either the form window or a ‘read-only’ version of the form window.



#### Complete your code in the cell below

The code cell below contains the imports you will need; the top level structure for the three classes to get you started; and the execution code at the bottom. 

In [17]:
# imports
import tkinter as tk
from tkinter import filedialog, messagebox
import datetime # one module for working with dates and times

# The MainWindow class creates a custom GUI window based on the tkinter window (tk.Tk)
# It has an __init__() method, and three additional methods (new_note(), open_notebook(), and save_notebook())
# These methods correspond to new, open, and save buttons in the window.
# The new_note method calls the NoteForm class to create a new note form top level window.

class MainWindow(tk.Tk):
    def __init__(self):  #initialize the main window
        super().__init__() # initialize it as a tkinter window
        
        self.geometry("600x400") # set the default window size
        self.title('Notebook') #set the default window title
        self.notes = []        

        # add additionallines to the __init__() function

        #create buttons
        tk.Button(self, text = 'New Note', command = self.new_note).pack()
        tk.Button(self, text = 'Open Notebook', command = self.open_notebook).pack()
        tk.Button(self, text = 'Save Notebook', command = self.save_notebook).pack() 
        # loads any notebook 
        self.load_notebook() 
        
        
        
    def new_note(self):
        NoteForm(self) # creates new note window

    def display_notes(self):   
        for widget in self.winfo_children():
            if isinstance(widget, tk.Button) and not widget.cget("text").startswith(("Open", "Save", "New")):
                widget.destroy()
        #displays notes as button 
        for note in self.notes:
            tk.Button(self, text = note['title'], command=lambda note = note: self.open_note(note)).pack() 

    def open_note(self, note):
        NoteForm(self, note)  
            

    def open_notebook(self):
        #opens a filedialog 
        filepath = filedialog.askopenfilename(filetypes= [("Text files", "*.txt"), ("All files", "*.*")])
        if filepath:
            with open(filepath, 'r') as file:
                #split the file content to separate notes and parse each note
                empty_notes = file.read().split('---\n')[1:]  
                self.notes = [self.parse_note_text(note) for note in empty_notes]
            self.display_notes()

    def parse_note_text(self, note_text):
        title, text, metadata = note_text.strip().split('|')
        return{'title': title, 'text': text, 'metadata': metadata}

    def save_notebook(self):
        #open a file dialog to choose where to save the notebook
        filepath = filedialog.asksaveasfile(mode='w', defaultextension=".txt", filetypes=[("Text files", "*.txt"), ("All files", "*.*")])
        if filepath:
            for note in self.notes:
                #write each note as a string separated by '---\n'
                filepath.write(f"---\n{note['title']}|{note['text']}|{note['metadata']}\n")
            filepath.close()
            messagebox.showinfo("Notebook has been saved.")

    def load_notebook(self):
        #placeholder to automatically load a default notebook file if exists
        pass
    
    def print_notes(self):
        for note in self.notes:
            print(f"Title: {note['title']}\nText: {note['text']}\nMetadata{note['metadata']}\n")
            
        

# the NoteForm() class creates a Toplevel window that is a note form containing fields for
# data entry for title, text, link, and tags. It also calculates a meta field with date, time, and timezone
# the Noteform class has an __init__() method, and a submit() method that is called by a submit button
# the class may contain additional methods to perform tasks like calculating the metadata, for example
# the submit method calls the MakeNote class that transforms the the entered data into a new note object.

class NoteForm(tk.Toplevel):
    
    def __init__(self, master, note =None): # initialize the new object
        super().__init__(master) # initialize it as a toplevel window

        # add additonal lines to set the new window's attributes and default parameters
        self.geometry("400x300") 
        
        self.title_entry = tk.Entry(self)
        self.title_entry.pack()
        self.text_entry = tk.Text(self, height = 10)
        self.text_entry.pack()

        #editing a previous note file
        if note:
            self.title_entry.insert(0, note['title'])
            self.text_entry.insert('1.0', note['text'])

        # create button
        tk.Button(self, text = 'SUBMIT', command=lambda: self.submit(note)).pack()

   
    def submit(self, note):
        # add lines to the submit method
        title = self.title_entry.get() # gathers info and updates
        text = self.text_entry.get('1.0', 'end').strip()
        metadata = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")

        if note: 
            note_index = self.master.notes.index(note)
            self.master.notes[note_index] = {'title': title, 'text': text, 'metadata': metadata}
        else:
            new_note = {'title': title, 'text': text, 'metadata': metadata}
            self.master.notes.append(new_note)

        self.master.display_notes() # displays
        self.destroy() # closes window
        

    
# The MakeNote class takes a dictionary containing the data entered into the form window,
# and transforms it into a new note object.
# At present the note objects have attributes but no methods.

class MakeNote:
    def __init__(self, note_dict):
        self.title = note_dict['title']
        self.text = note_dict['text']
        self.metadata = note_dict['metadata'] 
        
    
 

# main execution

if __name__ == '__main__':
    
    main_window = MainWindow() # this creates a notebook / main window called main_window. You may change the name if you want

    main_window.mainloop()

#### Print your three notes below

In [18]:
# print your notes here
with open('project_2note.txt', 'r') as file: # opens file name, reads and prints below
    notes = file.read()
    print(notes) 


---
project_2note|1) try not to overcomplicate python. I try to think of it as simply as input output. This helps me simplify whatI want done and allows me to check off tasks that need to be done efficiently. 

2) study using visuals, whether it be from websites,    example codes from lecture, or even youtube videos.Theseexamples will help break down concepts and see them in action.

3) write easy to read/understandable notes for urself.  This helps create markers for not only urself. But others, so that you understand where your thought process wasfor ur code.|2024-03-31 22:28:46

